In [1]:
#importing all necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandasql import sqldf
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import json
from pandas.io.json import json_normalize
import requests
from selenium import webdriver
import time
import requests
import lxml.html as lh
import bs4 as bs
import urllib.request
from shapely.geometry import Point
!pip install folium

In [2]:
#retreive data from  https://en.wikipedia.org/wiki/List_of_postal_codes_in_Nepal,
url = "https://github.com/unknown095/IDS-projects/blob/main/Book%20(1).csv"
res = requests.get(url)
soup = bs.BeautifulSoup(res.content,'lxml')
soup

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<link href="https://github.githubassets.com" rel="dns-prefetch"/>
<link href="https://avatars.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
<link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/frameworks-1640f59f9fa862ecff09f69ec92c615d.css" integrity="sha512-FkD1n5+oYuz/CfaeySxhXdkrLsPnfzdSiMgZPnzoNpv309HkRvJ9Wd+9OuLgshNi5h+TUl02PT1Tfdoi3G6kJg==" media="all" rel="stylesheet"/>
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/site-07eb9a80f5cb23c5e84ad18f73d0eb43.css" integrity="sha512-B+uagPXLI8XoStGPc9DrQ9xVWfBS0VvAKvYTQopaizdxo8T46TCtAVE9sY15FQ0kseA1LB3V5GKklDGuwjqCHg==" media="all" rel="stylesheet"/>
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/behaviors-060bad9e1c3288a40b14376c2e737e0b.css" integr

In [3]:
table = soup.find_all('table')[0]                   #store as table
df = pd.read_html(str(table))                       #create df
data = pd.read_json(df[0].to_json(orient='records'))
data.head()

,Unnamed: 0,Column1,Location,Ward,City,latitude,longitude,Area(sq km),Population Density (per sq km)
0,NaN,0,Naxal,Ward 1,Kathmandu,27.712678,85.328703,1.3818,5795
1,NaN,1,Lazimpat,Ward 2,Kathmandu,27.721508,85.320765,0.8181,16438
2,NaN,2,Maharajgunj,Ward 3,Kathmandu,27.733079,85.328807,3.0854,11300
3,NaN,3,"Baluwatar,",Ward 4,Kathmandu,27.724603,85.331017,3.3498,14139
4,NaN,4,Tangal,Ward 5,Kathmandu,27.717231,85.330449,0.7875,23263


In [4]:
data1=data[["Location","Ward","City","latitude","longitude","Population Density (per sq km)"]]

In [5]:
data1.isnull().sum()

Location                          1
Ward                              0
City                              0
latitude                          1
longitude                         1
Population Density (per sq km)    0
dtype: int64

In [6]:
data1=data1.replace("",0)

In [7]:
data1.isna().sum()

Location                          1
Ward                              0
City                              0
latitude                          1
longitude                         1
Population Density (per sq km)    0
dtype: int64

In [8]:
data1=data1.dropna()

In [9]:
data1.isnull().sum()

Location                          0
Ward                              0
City                              0
latitude                          0
longitude                         0
Population Density (per sq km)    0
dtype: int64

In [10]:
data1.isna().sum()

Location                          0
Ward                              0
City                              0
latitude                          0
longitude                         0
Population Density (per sq km)    0
dtype: int64

In [11]:
data1

,Location,Ward,City,latitude,longitude,Population Density (per sq km)
0,Naxal,Ward 1,Kathmandu,27.712678,85.328703,5795
1,Lazimpat,Ward 2,Kathmandu,27.721508,85.320765,16438
2,Maharajgunj,Ward 3,Kathmandu,27.733079,85.328807,11300
3,"Baluwatar,",Ward 4,Kathmandu,27.724603,85.331017,14139
4,Tangal,Ward 5,Kathmandu,27.717231,85.330449,23263
5,Bouddha,Ward 6,Kathmandu,27.722053,85.362081,16726
6,Mitrapark,Ward 7,Kathmandu,27.713360,85.345041,29507
7,Jayabageshwori,Ward 8,Kathmandu,27.710336,85.344025,3680
8,Gaushala,Ward 9,Kathmandu,27.709706,85.346409,13742
9,New Baneshwor,Ward 10,Kathmandu,27.692621,85.338072,25455


In [12]:
data1.shape

(51, 6)

In [13]:
geodata=data1[["latitude","longitude"]]
geodata.head()

,latitude,longitude
0,27.712678,85.328703
1,27.721508,85.320765
2,27.733079,85.328807
3,27.724603,85.331017
4,27.717231,85.330449


In [14]:
data1['Coordinates']=list(zip(data1["latitude"],data1["longitude"]))
data1.head()

,Location,Ward,City,latitude,longitude,Population Density (per sq km),Coordinates
0,Naxal,Ward 1,Kathmandu,27.712678,85.328703,5795,"(27.7126782, 85.3287033)"
1,Lazimpat,Ward 2,Kathmandu,27.721508,85.320765,16438,"(27.7215082, 85.3207646)"
2,Maharajgunj,Ward 3,Kathmandu,27.733079,85.328807,11300,"(27.7330791, 85.3288072)"
3,"Baluwatar,",Ward 4,Kathmandu,27.724603,85.331017,14139,"(27.7246028, 85.3310167)"
4,Tangal,Ward 5,Kathmandu,27.717231,85.330449,23263,"(27.7172307, 85.3304488)"


In [15]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 52
Data columns (total 7 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Location                        51 non-null     object 
 1   Ward                            51 non-null     object 
 2   City                            51 non-null     object 
 3   latitude                        51 non-null     float64
 4   longitude                       51 non-null     float64
 5   Population Density (per sq km)  51 non-null     int64  
 6   Coordinates                     51 non-null     object 
dtypes: float64(2), int64(1), object(4)
memory usage: 2.4+ KB


In [16]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [17]:
import pandas, os, geopy
from geopy.geocoders import Nominatim

In [18]:
geolocator = Nominatim(user_agent="coursera")
address = 'Kathmandu'
try:
    location = geolocator.geocode(address)
    latitude = data1['latitude'].astype(float)
    longitude = data1['longitude'].astype(float)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(data1['latitude'],data1['longitude'], data1['Location']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

The geograpical coordinates of Kathmandu are 27.708317, 85.3205817.


In [19]:
address = 'Kathmandu, Nepal'

geolocator = Nominatim(user_agent='coursera')
location = geolocator.geocode(address)
latitude = data1['latitude']
longitude = data1['longitude']
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Kathmandu are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of the City of Kathmandu are 27.708317, 85.3205817.


In [20]:
import folium

In [21]:
data1.head()

,Location,Ward,City,latitude,longitude,Population Density (per sq km),Coordinates
0,Naxal,Ward 1,Kathmandu,27.712678,85.328703,5795,"(27.7126782, 85.3287033)"
1,Lazimpat,Ward 2,Kathmandu,27.721508,85.320765,16438,"(27.7215082, 85.3207646)"
2,Maharajgunj,Ward 3,Kathmandu,27.733079,85.328807,11300,"(27.7330791, 85.3288072)"
3,"Baluwatar,",Ward 4,Kathmandu,27.724603,85.331017,14139,"(27.7246028, 85.3310167)"
4,Tangal,Ward 5,Kathmandu,27.717231,85.330449,23263,"(27.7172307, 85.3304488)"


In [22]:
map_kat=folium.Map(location=[(latitude),(longitude)],zoom_start=10)

In [23]:
for lat, lng, Location, City in zip(data1['latitude'], data1['longitude'], data1['Location'], data1['City']):
    label = '{}, {}'.format(City, Location)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_kat)  
    
map_kat

In [24]:
CLIENT_ID = 'PFMCEGVYKJ4VWFKRNXJKHUXUEAEFU0DD1RJUPLNRHSEJXLIX' # your Foursquare ID
CLIENT_SECRET = 'NSEGOTA1QSUTNLI02ZHR5QP4B1SFSB0VC0CQMD0BFSHOIEOG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PFMCEGVYKJ4VWFKRNXJKHUXUEAEFU0DD1RJUPLNRHSEJXLIX
CLIENT_SECRET:NSEGOTA1QSUTNLI02ZHR5QP4B1SFSB0VC0CQMD0BFSHOIEOG


In [25]:
merge_kat=data1[data1['City'].str.contains('Kathmandu')]
kat_data = merge_kat.reset_index(drop=True)
kat_data.head()

,Location,Ward,City,latitude,longitude,Population Density (per sq km),Coordinates
0,Naxal,Ward 1,Kathmandu,27.712678,85.328703,5795,"(27.7126782, 85.3287033)"
1,Lazimpat,Ward 2,Kathmandu,27.721508,85.320765,16438,"(27.7215082, 85.3207646)"
2,Maharajgunj,Ward 3,Kathmandu,27.733079,85.328807,11300,"(27.7330791, 85.3288072)"
3,"Baluwatar,",Ward 4,Kathmandu,27.724603,85.331017,14139,"(27.7246028, 85.3310167)"
4,Tangal,Ward 5,Kathmandu,27.717231,85.330449,23263,"(27.7172307, 85.3304488)"


In [26]:
map_kat = folium.Map(location=[latitude, longitude], zoom_start=12.5)


In [27]:
for lat, lng, city, location in zip(kat_data['latitude'], kat_data['longitude'], kat_data['City'], kat_data['Location']):
    label = '{}, {}'.format(location, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_kat)  
    
map_kat

In [28]:
kat_data.head()

,Location,Ward,City,latitude,longitude,Population Density (per sq km),Coordinates
0,Naxal,Ward 1,Kathmandu,27.712678,85.328703,5795,"(27.7126782, 85.3287033)"
1,Lazimpat,Ward 2,Kathmandu,27.721508,85.320765,16438,"(27.7215082, 85.3207646)"
2,Maharajgunj,Ward 3,Kathmandu,27.733079,85.328807,11300,"(27.7330791, 85.3288072)"
3,"Baluwatar,",Ward 4,Kathmandu,27.724603,85.331017,14139,"(27.7246028, 85.3310167)"
4,Tangal,Ward 5,Kathmandu,27.717231,85.330449,23263,"(27.7172307, 85.3304488)"


In [29]:
near_latitude = kat_data.loc[0, 'latitude'] # neighbourhood latitude value
near_longitude = kat_data.loc[0, 'longitude'] # neighbourhood longitude value

near_name = kat_data.loc[0, 'Location'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(near_name, 
                                                               near_latitude, 
                                                               near_longitude))

Latitude and longitude values of Naxal are 27.7126782, 85.3287033.


In [30]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    near_latitude, 
    near_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=PFMCEGVYKJ4VWFKRNXJKHUXUEAEFU0DD1RJUPLNRHSEJXLIX&client_secret=NSEGOTA1QSUTNLI02ZHR5QP4B1SFSB0VC0CQMD0BFSHOIEOG&v=20180605&ll=27.7126782,85.3287033&radius=500&limit=100'

In [31]:
result=requests.get(url).json()

In [32]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [33]:
venues = result['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

nearby_venues

<ipython-input-33-fd1d8e65eb44>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.address,venue.location.crossStreet,venue.location.city,venue.location.state,venue.location.postalCode
0,e-0-4fe5ab04e4b04d88c5ff84ab-0,0,"[{'summary': 'This spot is popular', 'type': '...",4fe5ab04e4b04d88c5ff84ab,Wunjala Moskva,27.714212,85.328233,"[{'label': 'display', 'lat': 27.71421238783959...",176,NP,नेपाल,[नेपाल],"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",0,[],NaN,NaN,NaN,NaN,NaN
1,e-0-5045e081e4b03986a6829e90-1,0,"[{'summary': 'This spot is popular', 'type': '...",5045e081e4b03986a6829e90,Brian's Grill House,27.708351,85.327781,"[{'label': 'display', 'lat': 27.70835114829886...",490,NP,नेपाल,"[Sunrise Bizz Park, 9th Floor (Dili Bazar), का...","[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",0,[],"Sunrise Bizz Park, 9th Floor",Dili Bazar,काठमाडौं,Central Region,NaN
2,e-0-4fe6b1a9e4b09fd373dce83e-2,0,"[{'summary': 'This spot is popular', 'type': '...",4fe6b1a9e4b09fd373dce83e,Tamaslaya,27.714850,85.326449,"[{'label': 'display', 'lat': 27.71485006554679...",328,NP,नेपाल,"[Naxal, नेपाल]","[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",0,[],Naxal,NaN,NaN,NaN,NaN
3,e-0-4c50e642bd099521c2b95b5e-3,0,"[{'summary': 'This spot is popular', 'type': '...",4c50e642bd099521c2b95b5e,Big Movies,27.709667,85.326619,"[{'label': 'display', 'lat': 27.70966704712293...",393,NP,नेपाल,"[Pashupati Road (City Center Mall), काठमाडौं 4...","[{'id': '4bf58dd8d48988d180941735', 'name': 'M...",0,[],Pashupati Road,City Center Mall,काठमाडौं,Central Region,44600
4,e-0-4cf8514877d760fce9844f33-4,0,"[{'summary': 'This spot is popular', 'type': '...",4cf8514877d760fce9844f33,City Centre,27.709703,85.326651,"[{'label': 'display', 'lat': 27.70970318422972...",388,NP,नेपाल,"[kamalpokhari, काठमाडौं, नेपाल]","[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",0,[],kamalpokhari,NaN,काठमाडौं,Central Region,NaN
5,e-0-50cda174e4b0c28fcee32cce-5,0,"[{'summary': 'This spot is popular', 'type': '...",50cda174e4b0c28fcee32cce,Rage Fitness,27.714958,85.326426,"[{'label': 'display', 'lat': 27.71495849554930...",338,NP,नेपाल,"[Narayan Chaur (Naxal), काठमाडौं, नेपाल]","[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",0,[],Narayan Chaur,Naxal,काठमाडौं,Central Region,NaN


In [34]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wunjala Moskva,Restaurant,27.714212,85.328233
1,Brian's Grill House,American Restaurant,27.708351,85.327781
2,Tamaslaya,Café,27.714850,85.326449
3,Big Movies,Multiplex,27.709667,85.326619
4,City Centre,Shopping Mall,27.709703,85.326651


In [35]:
nearby_venues['categories'].describe()

count              6
unique             6
top       Restaurant
freq               1
Name: categories, dtype: object

In [36]:
nearby_venues['categories'].sum().split()

['RestaurantAmerican', 'RestaurantCaféMultiplexShopping', 'MallGym']

In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [38]:
kat_data.head()

,Location,Ward,City,latitude,longitude,Population Density (per sq km),Coordinates
0,Naxal,Ward 1,Kathmandu,27.712678,85.328703,5795,"(27.7126782, 85.3287033)"
1,Lazimpat,Ward 2,Kathmandu,27.721508,85.320765,16438,"(27.7215082, 85.3207646)"
2,Maharajgunj,Ward 3,Kathmandu,27.733079,85.328807,11300,"(27.7330791, 85.3288072)"
3,"Baluwatar,",Ward 4,Kathmandu,27.724603,85.331017,14139,"(27.7246028, 85.3310167)"
4,Tangal,Ward 5,Kathmandu,27.717231,85.330449,23263,"(27.7172307, 85.3304488)"


In [39]:
venues = getNearbyVenues(names=kat_data['Location'],latitudes=kat_data['latitude'],longitudes=kat_data['longitude'])

Naxal
Lazimpat
Maharajgunj
Baluwatar,
Tangal
Bouddha
Mitrapark
Jayabageshwori
Gaushala
New Baneshwor
Tripureshwor
Kalimati
Kalanki
Swayambhu
Sohrakhutte
Chhetrapati
Bhimsenthan
Om Bahal
Lainchaur
Purano Buspark
Dillibazar
Gyaneshwor
Bhimsengola
Koteshwor


In [40]:
print(venues.shape)
venues.head()

(250, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Naxal,27.712678,85.328703,Wunjala Moskva,27.714212,85.328233,Restaurant
1,Naxal,27.712678,85.328703,Brian's Grill House,27.708351,85.327781,American Restaurant
2,Naxal,27.712678,85.328703,Tamaslaya,27.714850,85.326449,Café
3,Naxal,27.712678,85.328703,Big Movies,27.709667,85.326619,Multiplex
4,Naxal,27.712678,85.328703,City Centre,27.709703,85.326651,Shopping Mall


In [41]:
venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Baluwatar,",8,8,8,8,8,8
Bhimsengola,4,4,4,4,4,4
Bhimsenthan,17,17,17,17,17,17
Bouddha,21,21,21,21,21,21
Chhetrapati,28,28,28,28,28,28
Dillibazar,11,11,11,11,11,11
Gaushala,4,4,4,4,4,4
Gyaneshwor,4,4,4,4,4,4
Jayabageshwori,4,4,4,4,4,4


In [42]:
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

There are 68 unique categories.


In [43]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighbourhood'] = venues['Neighbourhood'] 

onehot.head()

,American Restaurant,Asian Restaurant,Athletics & Sports,Bakery,Bar,Bed & Breakfast,Beer Garden,Bistro,Boarding House,Breakfast Spot,...,Snack Place,Soccer Stadium,Spanish Restaurant,Spiritual Center,Temple,Tibetan Restaurant,Tourist Information Center,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Neighbourhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Naxal
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Naxal
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Naxal
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Naxal
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Naxal


In [44]:
onehot.shape

(250, 69)

In [45]:
grouped = onehot.groupby('Neighbourhood').mean().reset_index()
grouped

,Neighbourhood,American Restaurant,Asian Restaurant,Athletics & Sports,Bakery,Bar,Bed & Breakfast,Beer Garden,Bistro,Boarding House,...,Shopping Mall,Snack Place,Soccer Stadium,Spanish Restaurant,Spiritual Center,Temple,Tibetan Restaurant,Tourist Information Center,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,"Baluwatar,",0.000000,0.000000,0.00,0.125000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bhimsengola,0.000000,0.000000,0.25,0.250000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Bhimsenthan,0.000000,0.058824,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Bouddha,0.000000,0.095238,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.047619,0.047619,0.000000,0.047619,0.000000,0.047619,0.000000
4,Chhetrapati,0.000000,0.142857,0.00,0.000000,0.071429,0.035714,0.035714,0.035714,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.035714,0.000000
5,Dillibazar,0.090909,0.090909,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Gaushala,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000
7,Gyaneshwor,0.250000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Jayabageshwori,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Kalanki,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [46]:
grouped.shape

(24, 69)

In [47]:
num_top_venues = 5

for hood in grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Baluwatar,----
            venue  freq
0      Restaurant  0.25
1  Breakfast Spot  0.25
2            Café  0.25
3           Diner  0.12
4          Bakery  0.12


----Bhimsengola----
                 venue  freq
0                 Café  0.50
1   Athletics & Sports  0.25
2               Bakery  0.25
3  American Restaurant  0.00
4               Lounge  0.00


----Bhimsenthan----
                     venue  freq
0                     Café  0.24
1            Historic Site  0.24
2               Restaurant  0.18
3                    Hotel  0.06
4  Comfort Food Restaurant  0.06


----Bouddha----
              venue  freq
0              Café  0.19
1             Hotel  0.10
2  Asian Restaurant  0.10
3               Gym  0.10
4       Coffee Shop  0.10


----Chhetrapati----
                venue  freq
0          Restaurant  0.21
1    Asian Restaurant  0.14
2  Italian Restaurant  0.11
3                 Bar  0.07
4         Coffee Shop  0.07


----Dillibazar----
                 venue  freq
0      

In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Neighbourhood_venues_sorted = pd.DataFrame(columns=columns)
Neighbourhood_venues_sorted['Neighbourhood'] = grouped['Neighbourhood']

for ind in np.arange(grouped.shape[0]):
    Neighbourhood_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

Neighbourhood_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Baluwatar,",Restaurant,Breakfast Spot,Café,Diner,Bakery,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club
1,Bhimsengola,Café,Athletics & Sports,Bakery,American Restaurant,Lounge,Pizza Place,Park,Paper / Office Supplies Store,Multiplex,Mobile Phone Shop
2,Bhimsenthan,Café,Historic Site,Restaurant,Hotel,Comfort Food Restaurant,Coffee Shop,Road,Plaza,Asian Restaurant,Jazz Club
3,Bouddha,Café,Hotel,Asian Restaurant,Gym,Coffee Shop,Buddhist Temple,Indian Restaurant,Fast Food Restaurant,Chinese Restaurant,Restaurant
4,Chhetrapati,Restaurant,Asian Restaurant,Italian Restaurant,Bar,Coffee Shop,Café,Hookah Bar,Halal Restaurant,Eastern European Restaurant,Sandwich Place
5,Dillibazar,Café,American Restaurant,Shopping Mall,Asian Restaurant,Multiplex,Electronics Store,Dumpling Restaurant,Hotel,Comfort Food Restaurant,Department Store
6,Gaushala,Hotel,Temple,Farm,Boarding House,American Restaurant,Juice Bar,Paper / Office Supplies Store,Multiplex,Mobile Phone Shop,Lounge
7,Gyaneshwor,American Restaurant,Multiplex,Indian Restaurant,Shopping Mall,Jazz Club,Paper / Office Supplies Store,Mobile Phone Shop,Lounge,Juice Bar,Japanese Restaurant
8,Jayabageshwori,Historic Site,Park,Chaat Place,Mobile Phone Shop,American Restaurant,Juice Bar,Paper / Office Supplies Store,Multiplex,Lounge,Jazz Club
9,Kalanki,Café,Indian Restaurant,Paper / Office Supplies Store,Fast Food Restaurant,Department Store,American Restaurant,Mobile Phone Shop,Pizza Place,Park,Multiplex


In [50]:
kclusters = 5

grouped_clustering = grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 4, 0, 0, 0, 0, 4, 0, 3, 0])

In [51]:
Neighbourhood_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Baluwatar,",Restaurant,Breakfast Spot,Café,Diner,Bakery,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club
1,Bhimsengola,Café,Athletics & Sports,Bakery,American Restaurant,Lounge,Pizza Place,Park,Paper / Office Supplies Store,Multiplex,Mobile Phone Shop
2,Bhimsenthan,Café,Historic Site,Restaurant,Hotel,Comfort Food Restaurant,Coffee Shop,Road,Plaza,Asian Restaurant,Jazz Club
3,Bouddha,Café,Hotel,Asian Restaurant,Gym,Coffee Shop,Buddhist Temple,Indian Restaurant,Fast Food Restaurant,Chinese Restaurant,Restaurant
4,Chhetrapati,Restaurant,Asian Restaurant,Italian Restaurant,Bar,Coffee Shop,Café,Hookah Bar,Halal Restaurant,Eastern European Restaurant,Sandwich Place


In [52]:
kat_data=kat_data.rename(columns={'Location':'Neighbourhood'}).head()
kat_data.head()

,Neighbourhood,Ward,City,latitude,longitude,Population Density (per sq km),Coordinates
0,Naxal,Ward 1,Kathmandu,27.712678,85.328703,5795,"(27.7126782, 85.3287033)"
1,Lazimpat,Ward 2,Kathmandu,27.721508,85.320765,16438,"(27.7215082, 85.3207646)"
2,Maharajgunj,Ward 3,Kathmandu,27.733079,85.328807,11300,"(27.7330791, 85.3288072)"
3,"Baluwatar,",Ward 4,Kathmandu,27.724603,85.331017,14139,"(27.7246028, 85.3310167)"
4,Tangal,Ward 5,Kathmandu,27.717231,85.330449,23263,"(27.7172307, 85.3304488)"


In [53]:
Neighbourhood_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Baluwatar,",Restaurant,Breakfast Spot,Café,Diner,Bakery,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club
1,Bhimsengola,Café,Athletics & Sports,Bakery,American Restaurant,Lounge,Pizza Place,Park,Paper / Office Supplies Store,Multiplex,Mobile Phone Shop
2,Bhimsenthan,Café,Historic Site,Restaurant,Hotel,Comfort Food Restaurant,Coffee Shop,Road,Plaza,Asian Restaurant,Jazz Club
3,Bouddha,Café,Hotel,Asian Restaurant,Gym,Coffee Shop,Buddhist Temple,Indian Restaurant,Fast Food Restaurant,Chinese Restaurant,Restaurant
4,Chhetrapati,Restaurant,Asian Restaurant,Italian Restaurant,Bar,Coffee Shop,Café,Hookah Bar,Halal Restaurant,Eastern European Restaurant,Sandwich Place


In [54]:

merged = kat_data

# add clustering labels

#merged['Cluster Labels'] = kmeans.labels_

Neighbourhood_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

merged = merged.join(Neighbourhood_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood' ,how='inner')
uno=merged
merged.head()


,Neighbourhood,Ward,City,latitude,longitude,Population Density (per sq km),Coordinates,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Naxal,Ward 1,Kathmandu,27.712678,85.328703,5795,"(27.7126782, 85.3287033)",0,American Restaurant,Shopping Mall,Restaurant,Multiplex,Gym,Café,Paper / Office Supplies Store,Mobile Phone Shop,Lounge,Juice Bar
1,Lazimpat,Ward 2,Kathmandu,27.721508,85.320765,16438,"(27.7215082, 85.3207646)",4,Hotel,Café,Restaurant,Asian Restaurant,Hotel Bar,Himalayan Restaurant,Ice Cream Shop,Jazz Club,Vietnamese Restaurant,Pub
2,Maharajgunj,Ward 3,Kathmandu,27.733079,85.328807,11300,"(27.7330791, 85.3288072)",0,Asian Restaurant,French Restaurant,Breakfast Spot,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Plaza
3,"Baluwatar,",Ward 4,Kathmandu,27.724603,85.331017,14139,"(27.7246028, 85.3310167)",0,Restaurant,Breakfast Spot,Café,Diner,Bakery,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club
4,Tangal,Ward 5,Kathmandu,27.717231,85.330449,23263,"(27.7172307, 85.3304488)",0,Restaurant,Asian Restaurant,Café,Department Store,Snack Place,Park,Diner,Coffee Shop,American Restaurant,Bar


In [55]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['latitude'], merged['longitude'], merged['Neighbourhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [56]:
merged.loc[merged['Cluster Labels'] == 0, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Ward,Population Density (per sq km),Coordinates,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ward 1,5795,"(27.7126782, 85.3287033)",0,American Restaurant,Shopping Mall,Restaurant,Multiplex,Gym,Café,Paper / Office Supplies Store,Mobile Phone Shop,Lounge,Juice Bar
2,Ward 3,11300,"(27.7330791, 85.3288072)",0,Asian Restaurant,French Restaurant,Breakfast Spot,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Plaza
3,Ward 4,14139,"(27.7246028, 85.3310167)",0,Restaurant,Breakfast Spot,Café,Diner,Bakery,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club
4,Ward 5,23263,"(27.7172307, 85.3304488)",0,Restaurant,Asian Restaurant,Café,Department Store,Snack Place,Park,Diner,Coffee Shop,American Restaurant,Bar


In [57]:

merged.loc[merged['Cluster Labels'] == 1, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Ward,Population Density (per sq km),Coordinates,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [58]:
merged.loc[merged['Cluster Labels'] == 2, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Ward,Population Density (per sq km),Coordinates,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [59]:
merged.loc[merged['Cluster Labels'] == 3, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Ward,Population Density (per sq km),Coordinates,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [60]:
merged.loc[merged['Cluster Labels'] == 4, merged.columns[[1] + list(range(5, merged.shape[1]))]]


,Ward,Population Density (per sq km),Coordinates,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Ward 2,16438,"(27.7215082, 85.3207646)",4,Hotel,Café,Restaurant,Asian Restaurant,Hotel Bar,Himalayan Restaurant,Ice Cream Shop,Jazz Club,Vietnamese Restaurant,Pub



#### XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

In [61]:
merge_lat=data1[data1['City'].str.contains('Lalitpur')]
lat_data = merge_lat.reset_index(drop=True)
lat_data=lat_data.rename(columns={'Location':'Neighbourhood'}).head()
lat_data

,Neighbourhood,Ward,City,latitude,longitude,Population Density (per sq km),Coordinates
0,Sanepa,Ward 2,Lalitpur,27.683772,85.309353,17148,"(27.6837719, 85.309353)"
1,Pulchowk,Ward 3,Lalitpur,27.678357,85.320608,8522,"(27.6783567, 85.3206084)"
2,Jawalakhel,Ward 4,Lalitpur,27.672440,85.313376,7538,"(27.6724402, 85.3133757)"
3,Lagankhel,Ward 5,Lalitpur,27.666873,85.323636,8387,"(27.6668726, 85.3236357)"
4,Kanibahal,"Ward 6, 7 and 8",Lalitpur,27.666727,85.328007,89212,"(27.6667273, 85.3280068)"


In [62]:
# create map of Toronto using latitude and longitude values
map_lat = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, city, neighbourhood in zip(lat_data['latitude'], lat_data['longitude'], lat_data['City'], lat_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_lat)  
    
map_lat

In [63]:
lat_data.loc[0, 'Neighbourhood']

'Sanepa'

In [64]:
neighbourhood_latitude = lat_data.loc[0, 'latitude'] # neighbourhood latitude value
neighbourhood_longitude = lat_data.loc[0, 'longitude'] # neighbourhood longitude value

neighbourhood_name = lat_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Sanepa are 27.6837719, 85.309353.


In [65]:
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=PFMCEGVYKJ4VWFKRNXJKHUXUEAEFU0DD1RJUPLNRHSEJXLIX&client_secret=NSEGOTA1QSUTNLI02ZHR5QP4B1SFSB0VC0CQMD0BFSHOIEOG&v=20180605&ll=27.6837719,85.309353&radius=500&limit=100'

In [66]:
results = requests.get(url).json()

In [67]:

venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

nearby_venues

<ipython-input-67-baa960f98222>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.address,venue.location.crossStreet,venue.location.postalCode,venue.location.city,venue.location.state
0,e-0-4feae739e4b03169143fa2d1-0,0,"[{'summary': 'This spot is popular', 'type': '...",4feae739e4b03169143fa2d1,Top Of The World,27.680864,85.310422,"[{'label': 'display', 'lat': 27.680864, 'lng':...",340,NP,नेपाल,[नेपाल],"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],NaN,NaN,NaN,NaN,NaN
1,e-0-50a7599ce4b0e4f42d17d0fa-1,0,"[{'summary': 'This spot is popular', 'type': '...",50a7599ce4b0e4f42d17d0fa,Momotarou Restaurant,27.681818,85.305657,"[{'label': 'display', 'lat': 27.68181777366247...",424,NP,नेपाल,"[Milap Rd. (None), Patan 44600, नेपाल]","[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",0,[],Milap Rd.,None,44600,Pātan,Central Region
2,e-0-52b57a34498e594be4c93a43-2,0,"[{'summary': 'This spot is popular', 'type': '...",52b57a34498e594be4c93a43,One Ugly Mug Coffee Bar,27.680893,85.310151,"[{'label': 'display', 'lat': 27.68089307230889...",329,NP,नेपाल,[नेपाल],"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],NaN,NaN,NaN,NaN,NaN
3,e-0-4e245b4dd22d0a3f5a149a0c-3,0,"[{'summary': 'This spot is popular', 'type': '...",4e245b4dd22d0a3f5a149a0c,Bu Keba The Organic Village Restaurant,27.683407,85.312963,"[{'label': 'display', 'lat': 27.68340678850808...",358,NP,नेपाल,"[Kupondole, Lalitpur, नेपाल]","[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",0,[],Kupondole,NaN,NaN,Pātan,Central Region
4,e-0-4e24275f1f6effe84f8bd77e-4,0,"[{'summary': 'This spot is popular', 'type': '...",4e24275f1f6effe84f8bd77e,Summit Hotel,27.685633,85.311841,"[{'label': 'display', 'lat': 27.68563274643064...",321,NP,नेपाल,"[Kupondole Heights Rd (Maitri Marg), Lalitpur ...","[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",0,[],Kupondole Heights Rd,Maitri Marg,44700,Pātan,Central Region
5,e-0-4c38a49a1a38ef3bb1af9221-5,0,"[{'summary': 'This spot is popular', 'type': '...",4c38a49a1a38ef3bb1af9221,"International Club, Sanepa",27.682870,85.307396,"[{'label': 'display', 'lat': 27.68286958445193...",217,NP,नेपाल,"[Sanepa Chowk, काठमाडौं, नेपाल]","[{'id': '4bf58dd8d48988d15e941735', 'name': 'P...",0,[],Sanepa Chowk,NaN,NaN,काठमाडौं,Central Region
6,e-0-4de9c9ae8130115d6bcde96a-6,0,"[{'summary': 'This spot is popular', 'type': '...",4de9c9ae8130115d6bcde96a,Black Pepper Cafe & Pub,27.683670,85.311736,"[{'label': 'display', 'lat': 27.68367010610196...",235,NP,नेपाल,"[Kupondole Height (Jhamshikhel), Lalitpur, नेपाल]","[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",0,[],Kupondole Height,Jhamshikhel,NaN,Pātan,Central Region
7,e-0-4eb7d794d3e3644ca73811c5-7,0,"[{'summary': 'This spot is popular', 'type': '...",4eb7d794d3e3644ca73811c5,Café Brown Sugar,27.682058,85.311012,"[{'label': 'display', 'lat': 27.68205833435058...",251,NP,नेपाल,"[Jhamaikhel, Lalitpur, नेपाल]","[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",0,[],Jhamaikhel,NaN,NaN,Pātan,Central Region
8,e-0-519b6c4a498ef838dde76511-8,0,"[{'summary': 'This spot is popular', 'type': '...",519b6c4a498ef838dde76511,Prismatic Futsal,27.680975,85.306768,"[{'label': 'display', 'lat': 27.68097531021327...",402,NP,नेपाल,[नेपाल],"[{'id': '4cce455aebf7b749d5e191f5', 'name': 'S...",0,[],NaN,NaN,NaN,NaN,NaN
9,e-0-582474662db9f377b9f0b7ba-9,0,"[{'summary': 'This spot is popular', 'type': '...",582474662db9f377b9f0b7ba,Nepa Beanz,27.687440,85.310630,"[{'label': 'display', 'lat': 27.68744, 'lng': ...",427,NP,नेपाल,"[Patan 44600, नेपाल]","[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",0,[],NaN,NaN,44600,Pātan,Central Region


In [68]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Top Of The World,Coffee Shop,27.680864,85.310422
1,Momotarou Restaurant,Japanese Restaurant,27.681818,85.305657
2,One Ugly Mug Coffee Bar,Coffee Shop,27.680893,85.310151
3,Bu Keba The Organic Village Restaurant,Restaurant,27.683407,85.312963
4,Summit Hotel,Hotel,27.685633,85.311841


In [69]:
nearby_venues['categories'].describe()

count             11
unique             8
top       Restaurant
freq               2
Name: categories, dtype: object

In [70]:
venues = getNearbyVenues(names=lat_data['Neighbourhood'],latitudes=lat_data['latitude'],longitudes=lat_data['longitude'])

Sanepa
Pulchowk
Jawalakhel
Lagankhel
Kanibahal


In [71]:
print(venues.shape)
venues.head()

(55, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Sanepa,27.683772,85.309353,Top Of The World,27.680864,85.310422,Coffee Shop
1,Sanepa,27.683772,85.309353,Momotarou Restaurant,27.681818,85.305657,Japanese Restaurant
2,Sanepa,27.683772,85.309353,One Ugly Mug Coffee Bar,27.680893,85.310151,Coffee Shop
3,Sanepa,27.683772,85.309353,Bu Keba The Organic Village Restaurant,27.683407,85.312963,Restaurant
4,Sanepa,27.683772,85.309353,Summit Hotel,27.685633,85.311841,Hotel


In [72]:
venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Jawalakhel,26,26,26,26,26,26
Kanibahal,4,4,4,4,4,4
Lagankhel,5,5,5,5,5,5
Pulchowk,9,9,9,9,9,9
Sanepa,11,11,11,11,11,11


In [73]:
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

There are 27 unique categories.


In [74]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighbourhood'] = venues['Neighbourhood'] 

onehot.head()

,Asian Restaurant,Athletics & Sports,Bakery,Burger Joint,Bus Station,Business Service,Café,Chinese Restaurant,Coffee Shop,Department Store,...,Latin American Restaurant,Malay Restaurant,Multiplex,Park,Pizza Place,Pool,Restaurant,Shopping Mall,Soccer Field,Neighbourhood
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Sanepa
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Sanepa
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Sanepa
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Sanepa
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Sanepa


In [75]:
onehot.shape

(55, 28)

In [76]:
grouped = onehot.groupby('Neighbourhood').mean().reset_index()
grouped

,Neighbourhood,Asian Restaurant,Athletics & Sports,Bakery,Burger Joint,Bus Station,Business Service,Café,Chinese Restaurant,Coffee Shop,...,Japanese Restaurant,Latin American Restaurant,Malay Restaurant,Multiplex,Park,Pizza Place,Pool,Restaurant,Shopping Mall,Soccer Field
0,Jawalakhel,0.076923,0.000000,0.076923,0.038462,0.00,0.000000,0.269231,0.076923,0.076923,...,0.000000,0.038462,0.038462,0.000000,0.000000,0.038462,0.000000,0.076923,0.0,0.000000
1,Kanibahal,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,Lagankhel,0.000000,0.000000,0.000000,0.000000,0.20,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.2,0.000000
3,Pulchowk,0.000000,0.000000,0.000000,0.000000,0.00,0.111111,0.222222,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.111111,0.111111,0.000000,0.000000,0.000000,0.0,0.000000
4,Sanepa,0.000000,0.090909,0.000000,0.000000,0.00,0.000000,0.181818,0.000000,0.181818,...,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.181818,0.0,0.090909


In [77]:
grouped.shape

(5, 28)

In [78]:
num_top_venues = 5

for hood in grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Jawalakhel----
                venue  freq
0                Café  0.27
1    Asian Restaurant  0.08
2              Bakery  0.08
3          Restaurant  0.08
4  Chinese Restaurant  0.08


----Kanibahal----
                 venue  freq
0                  Gym  0.25
1          Bus Station  0.25
2  Fried Chicken Joint  0.25
3                Hotel  0.25
4    Indian Restaurant  0.00


----Lagankhel----
               venue  freq
0              Hotel   0.2
1      Shopping Mall   0.2
2         Restaurant   0.2
3        Bus Station   0.2
4  Indian Restaurant   0.2


----Pulchowk----
              venue  freq
0              Café  0.22
1  Department Store  0.22
2  Business Service  0.11
3              Park  0.11
4         Multiplex  0.11


----Sanepa----
          venue  freq
0    Restaurant  0.18
1          Café  0.18
2   Coffee Shop  0.18
3  Soccer Field  0.09
4          Pool  0.09




In [79]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = grouped['Neighbourhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Jawalakhel,Café,Asian Restaurant,Bakery,Restaurant,Chinese Restaurant,Coffee Shop,Indian Restaurant,Pizza Place,Malay Restaurant,Latin American Restaurant
1,Kanibahal,Gym,Bus Station,Fried Chicken Joint,Hotel,Indian Restaurant,Shopping Mall,Restaurant,Pool,Pizza Place,Park
2,Lagankhel,Hotel,Shopping Mall,Restaurant,Bus Station,Indian Restaurant,Asian Restaurant,Pool,Pizza Place,Park,Multiplex
3,Pulchowk,Café,Department Store,Business Service,Park,Multiplex,Eastern European Restaurant,Indian Restaurant,Asian Restaurant,Italian Restaurant,Shopping Mall
4,Sanepa,Restaurant,Café,Coffee Shop,Soccer Field,Pool,Japanese Restaurant,Athletics & Sports,Hotel,Italian Restaurant,Shopping Mall


In [80]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 2, 1, 4, 0])

In [81]:
merged = lat_data



# add clustering labels

#merged['Cluster Labels'] = kmeans.labels_

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

merged = merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood' ,how='inner')
merged.head()
duo=merged

In [82]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12.5)
# set color scheme for the clustersx = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['latitude'], merged['longitude'], merged['Neighbourhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [83]:
merged.loc[merged['Cluster Labels'] == 0, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Ward,Population Density (per sq km),Coordinates,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ward 2,17148,"(27.6837719, 85.309353)",0,Restaurant,Café,Coffee Shop,Soccer Field,Pool,Japanese Restaurant,Athletics & Sports,Hotel,Italian Restaurant,Shopping Mall


In [84]:
merged.loc[merged['Cluster Labels'] == 1, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Ward,Population Density (per sq km),Coordinates,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Ward 5,8387,"(27.6668726, 85.3236357)",1,Hotel,Shopping Mall,Restaurant,Bus Station,Indian Restaurant,Asian Restaurant,Pool,Pizza Place,Park,Multiplex


In [85]:
merged.loc[merged['Cluster Labels'] == 2, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Ward,Population Density (per sq km),Coordinates,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,"Ward 6, 7 and 8",89212,"(27.6667273, 85.3280068)",2,Gym,Bus Station,Fried Chicken Joint,Hotel,Indian Restaurant,Shopping Mall,Restaurant,Pool,Pizza Place,Park


In [86]:
merged.loc[merged['Cluster Labels'] == 3, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Ward,Population Density (per sq km),Coordinates,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Ward 4,7538,"(27.6724402, 85.3133757)",3,Café,Asian Restaurant,Bakery,Restaurant,Chinese Restaurant,Coffee Shop,Indian Restaurant,Pizza Place,Malay Restaurant,Latin American Restaurant


In [87]:
merged.loc[merged['Cluster Labels'] == 4, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Ward,Population Density (per sq km),Coordinates,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Ward 3,8522,"(27.6783567, 85.3206084)",4,Café,Department Store,Business Service,Park,Multiplex,Eastern European Restaurant,Indian Restaurant,Asian Restaurant,Italian Restaurant,Shopping Mall


In [88]:
merged.head()

,Neighbourhood,Ward,City,latitude,longitude,Population Density (per sq km),Coordinates,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Sanepa,Ward 2,Lalitpur,27.683772,85.309353,17148,"(27.6837719, 85.309353)",0,Restaurant,Café,Coffee Shop,Soccer Field,Pool,Japanese Restaurant,Athletics & Sports,Hotel,Italian Restaurant,Shopping Mall
1,Pulchowk,Ward 3,Lalitpur,27.678357,85.320608,8522,"(27.6783567, 85.3206084)",4,Café,Department Store,Business Service,Park,Multiplex,Eastern European Restaurant,Indian Restaurant,Asian Restaurant,Italian Restaurant,Shopping Mall
2,Jawalakhel,Ward 4,Lalitpur,27.672440,85.313376,7538,"(27.6724402, 85.3133757)",3,Café,Asian Restaurant,Bakery,Restaurant,Chinese Restaurant,Coffee Shop,Indian Restaurant,Pizza Place,Malay Restaurant,Latin American Restaurant
3,Lagankhel,Ward 5,Lalitpur,27.666873,85.323636,8387,"(27.6668726, 85.3236357)",1,Hotel,Shopping Mall,Restaurant,Bus Station,Indian Restaurant,Asian Restaurant,Pool,Pizza Place,Park,Multiplex
4,Kanibahal,"Ward 6, 7 and 8",Lalitpur,27.666727,85.328007,89212,"(27.6667273, 85.3280068)",2,Gym,Bus Station,Fried Chicken Joint,Hotel,Indian Restaurant,Shopping Mall,Restaurant,Pool,Pizza Place,Park


#### XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

In [89]:
merge_bha=data1[data1['City'].str.contains('Bhaktapur')]
bha_data = merge_bha.reset_index(drop=True)
bha_data =bha_data.rename(columns={'Location':'Neighbourhood'}).head()
bha_data.head()

,Neighbourhood,Ward,City,latitude,longitude,Population Density (per sq km),Coordinates
0,mini Bus park,Ward 1,Bhaktapur,27.684014,85.422796,9901,"(27.68401389, 85.42279647)"
1,Duwakot,Ward 2,Bhaktapur,27.703937,85.455068,10553,"(27.70393659, 85.4550683)"
2,Nagarkot Road,Ward 4,Bhaktapur,27.688485,85.431673,8248,"(27.68848453, 85.43167256)"
3,Rammandir road,Ward 5,Bhaktapur,27.669910,85.430347,6425,"(27.66991041, 85.43034742)"
4,Incho - hanumanghat road,Ward 6,Bhaktapur,27.671735,85.435497,7212,"(27.67173475, 85.43549726)"


In [90]:
# create map of Toronto using latitude and longitude values
map_bha = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, city, neighbourhood in zip(bha_data['latitude'], bha_data['longitude'], bha_data['City'], bha_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_bha)  
    
map_bha

In [91]:
bha_data.loc[0, 'Neighbourhood']

'mini Bus park'

In [92]:
neighbourhood_latitude = bha_data.loc[0, 'latitude'] # neighbourhood latitude value
neighbourhood_longitude = bha_data.loc[0, 'longitude'] # neighbourhood longitude value

neighbourhood_name = bha_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of mini Bus park are 27.68401389, 85.42279647.


In [93]:
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=PFMCEGVYKJ4VWFKRNXJKHUXUEAEFU0DD1RJUPLNRHSEJXLIX&client_secret=NSEGOTA1QSUTNLI02ZHR5QP4B1SFSB0VC0CQMD0BFSHOIEOG&v=20180605&ll=27.68401389,85.42279647&radius=500&limit=100'

In [94]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '605ca1887fa3212e07f2033c'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Bhaktapur',
  'headerFullLocation': 'Bhaktapur',
  'headerLocationGranularity': 'city',
  'totalResults': 0,
  'suggestedBounds': {'ne': {'lat': 27.688513894500005,
    'lng': 85.42786872788716},
   'sw': {'lat': 27.679513885499993, 'lng': 85.41772421211283}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': []}]}}

In [95]:

venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

nearby_venues

<ipython-input-95-baa960f98222>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


""


In [96]:
venues

[]

In [97]:
# filter columns
#filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

KeyError: "None of [Index(['venue.name', 'venue.categories', 'venue.location.lat',\n       'venue.location.lng'],\n      dtype='object')] are in the [columns]"

In [ ]:
nearby_venues['categories'].describe()

In [ ]:
venues = getNearbyVenues(names=bha_data['Neighbourhood'],latitudes=bha_data['latitude'],longitudes=bha_data['longitude'])

In [ ]:
print(venues.shape)
venues.head()

In [ ]:
venues.groupby('Neighbourhood').count()

In [ ]:
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

In [ ]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighbourhood'] = venues['Neighbourhood'] 

onehot.head()

In [ ]:
onehot.shape

In [ ]:
grouped = onehot.groupby('Neighbourhood').mean().reset_index()
grouped

In [ ]:
grouped.shape

In [ ]:
num_top_venues = 5

for hood in grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = grouped['Neighbourhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

In [ ]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
merged = lat_data



# add clustering labels

#merged['Cluster Labels'] = kmeans.labels_

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

merged = merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood' ,how='inner')
merged=tres
merged.head()

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12.5)
# set color scheme for the clustersx = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['latitude'], merged['longitude'], merged['Neighbourhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
merged.loc[merged['Cluster Labels'] == 0, merged.columns[[1] + list(range(5, merged.shape[1]))]]

In [ ]:
merged.loc[merged['Cluster Labels'] == 1, merged.columns[[1] + list(range(5, merged.shape[1]))]]

In [ ]:
merged.loc[merged['Cluster Labels'] == 2, merged.columns[[1] + list(range(5, merged.shape[1]))]]

In [ ]:
merged.loc[merged['Cluster Labels'] == 3, merged.columns[[1] + list(range(5, merged.shape[1]))]]

In [ ]:
merged.loc[merged['Cluster Labels'] == 4, merged.columns[[1] + list(range(5, merged.shape[1]))]]

In [ ]:
merged.head()

#### XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

In [ ]:
compilation=compilation.append(uno,duo,tres)